In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib  # เอาไว้เซฟโมเดล

# --- 1. โหลดข้อมูลสอน (Training Data) ---
print("1. กำลังโหลดข้อมูลสอน...")
train_df = pd.read_csv('zeus_train.csv')

# เลือก Feature (ตัวแปรต้น) และ Target (สิ่งที่อยากทำนาย)
# เราเลือกตามผล EDA ที่วิเคราะห์มา
feature_cols = ['humidity', 'pressure', 'rain', 'uv', 'wind_speed', 'hour', 'is_day']
target_col = 'temp'

X_train = train_df[feature_cols]
y_train = train_df[target_col]

print(f"   - จำนวนข้อมูลสอน: {len(X_train)} แถว")
print(f"   - ตัวแปรที่ใช้เรียน: {feature_cols}")

# --- 2. สร้างและเทรนโมเดล (The Training) ---
print("\n2. กำลังปลุกเสก 'The Oracle' (Training Random Forest)...")
# n_estimators=100 คือสร้างต้นไม้ 100 ต้น
# random_state=42 คือล็อคผลให้เหมือนเดิมทุกครั้งที่รัน (เพื่อความชัวร์)
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1) 

model.fit(X_train, y_train) # <-- คำสั่งสอน AI อยู่ตรงนี้!
print("   ✅ เทรนเสร็จเรียบร้อย!")

# --- 3. บันทึกโมเดล (Save Model) ---
print("\n3. กำลังบันทึกโมเดลเก็บไว้...")
model_filename = 'zeus_oracle_model.pkl'
joblib.dump(model, model_filename)
print(f"   ✅ บันทึกไฟล์โมเดลชื่อ '{model_filename}' สำเร็จ! (เอาไฟล์นี้ไปใช้ทำเว็บต่อ)")

# --- 4. ตรวจสอบความฉลาดเบื้องต้น (Feature Importance) ---
print("\n4. ปัจจัยที่ส่งผลต่ออุณหภูมิ (Feature Importance):")
importances = model.feature_importances_
indices = np.argsort(importances)[::-1] # เรียงจากมากไปน้อย

for i in range(len(feature_cols)):
    curr_idx = indices[i]
    print(f"   อันดับ {i+1}: {feature_cols[curr_idx]} \t({importances[curr_idx]*100:.2f}%)")

print("-" * 30)
print("พร้อมสำหรับขั้นตอนต่อไป: Model Evaluation (สอบวัดผลกับข้อมูล Unseen)")

1. กำลังโหลดข้อมูลสอน...
   - จำนวนข้อมูลสอน: 39207 แถว
   - ตัวแปรที่ใช้เรียน: ['humidity', 'pressure', 'rain', 'uv', 'wind_speed', 'hour', 'is_day']

2. กำลังปลุกเสก 'The Oracle' (Training Random Forest)...
   ✅ เทรนเสร็จเรียบร้อย!

3. กำลังบันทึกโมเดลเก็บไว้...
   ✅ บันทึกไฟล์โมเดลชื่อ 'zeus_oracle_model.pkl' สำเร็จ! (เอาไฟล์นี้ไปใช้ทำเว็บต่อ)

4. ปัจจัยที่ส่งผลต่ออุณหภูมิ (Feature Importance):
   อันดับ 1: humidity 	(44.72%)
   อันดับ 2: uv 	(32.53%)
   อันดับ 3: pressure 	(12.66%)
   อันดับ 4: hour 	(7.65%)
   อันดับ 5: wind_speed 	(1.22%)
   อันดับ 6: rain 	(0.73%)
   อันดับ 7: is_day 	(0.49%)
------------------------------
พร้อมสำหรับขั้นตอนต่อไป: Model Evaluation (สอบวัดผลกับข้อมูล Unseen)


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import joblib

print("🚀 กำลังเริ่มสร้างโมเดลเสริม (Humidity, Rain, UV)...")

# 1. โหลดข้อมูล
df = pd.read_csv('zeus_train.csv')

# --- CLEANING RAIN DATA (สำคัญมาก!) ---
# ข้อมูลเดิมมีการเติม Mean (0.0457..) มา ทำให้คอมเข้าใจว่าฝนตกตลอดเวลา
# เราจะเคลียร์ค่าที่น้อยกว่า 0.05 ให้เป็น 0 (ฝนไม่ตก)
df['rain_clean'] = df['rain'].apply(lambda x: 0 if x <= 0.05 else 1) # 1 = ฝนตก, 0 = ไม่ตก
print(f"   - แก้ไขข้อมูลฝนเสร็จสิ้น (เจอวันฝนตกจริง: {df['rain_clean'].sum()} แถว)")

# เตรียม Feature พื้นฐาน (ต้องมี hour, is_day แล้ว)
if 'hour' not in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['hour'] = df['datetime'].dt.hour
    df['is_day'] = df['hour'].apply(lambda x: 1 if 6 <= x <= 18 else 0)

# --- Model 1: ทำนายความชื้น (Humidity) ---
print("1. กำลังเทรนโมเดล 'ความชื้น'...")
# ใช้ Rain แบบ Clean เป็นตัวช่วยทำนาย
X_hum = df[['temp', 'pressure', 'rain', 'uv', 'wind_speed', 'hour', 'is_day']]
y_hum = df['humidity']
hum_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
hum_model.fit(X_hum, y_hum)
joblib.dump(hum_model, 'zeus_humidity_model.pkl')

# --- Model 2: ทำนายโอกาสเกิดฝน (Rain Probability) ---
print("2. กำลังเทรนโมเดล 'โอกาสเกิดฝน' (Classifier)...")
# ใช้ Classifier เพื่อหา % โอกาสตก
# ตัด Rain ออกจาก input เพราะเราจะทำนาย Rain
X_rain = df[['temp', 'humidity', 'pressure', 'uv', 'wind_speed', 'hour', 'is_day']]
y_rain = df['rain_clean']
# class_weight='balanced' ช่วยแก้ปัญหาข้อมูลฝนตกน้อยกว่าฝนไม่ตก
rain_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1)
rain_model.fit(X_rain, y_rain)
joblib.dump(rain_model, 'zeus_rain_class_model.pkl')

# --- Model 3: ทำนาย UV ---
print("3. กำลังเทรนโมเดล 'UV'...")
X_uv = df[['temp', 'humidity', 'pressure', 'rain', 'wind_speed', 'hour', 'is_day']]
y_uv = df['uv']
uv_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
uv_model.fit(X_uv, y_uv)
joblib.dump(uv_model, 'zeus_uv_model.pkl')

print("✅ เสร็จสมบูรณ์! ได้ไฟล์โมเดลเพิ่ม 3 ไฟล์ พร้อมใช้งาน")

🚀 กำลังเริ่มสร้างโมเดลเสริม (Humidity, Rain, UV)...
   - แก้ไขข้อมูลฝนเสร็จสิ้น (เจอวันฝนตกจริง: 81 แถว)
1. กำลังเทรนโมเดล 'ความชื้น'...
2. กำลังเทรนโมเดล 'โอกาสเกิดฝน' (Classifier)...
3. กำลังเทรนโมเดล 'UV'...
✅ เสร็จสมบูรณ์! ได้ไฟล์โมเดลเพิ่ม 3 ไฟล์ พร้อมใช้งาน
